In [1]:
!pip install accelerate -U

In [2]:
!conda install -y gdown

Channels:
 - rapidsai
 - nvidia
 - conda-forge
 - defaults
 - pytorch
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [3]:
# https://drive.google.com/file/d/1-B27pzLWItwX5jzgVpf9c50yz0PRxug8/view?usp=drive_link
!gdown --id "1-B27pzLWItwX5jzgVpf9c50yz0PRxug8"

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-B27pzLWItwX5jzgVpf9c50yz0PRxug8
To: /kaggle/working/trainset_cmb.txt
100%|██████████████████████████████████████| 11.4M/11.4M [00:00<00:00, 66.1MB/s]


In [4]:
# https://drive.google.com/file/d/1-2e_zIZdQTNajebnVAbtzAIIELUnhotx/view?usp=sharing
!gdown --id "1-2e_zIZdQTNajebnVAbtzAIIELUnhotx"

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-2e_zIZdQTNajebnVAbtzAIIELUnhotx
To: /kaggle/working/valset_cmb.txt
100%|███████████████████████████████████████| 2.95M/2.95M [00:00<00:00, 186MB/s]


In [5]:
# https://drive.google.com/file/d/1m11t37h2bLU2wmd41aTEQREGWsFJGyfj/view?usp=sharing
!gdown --id "1m11t37h2bLU2wmd41aTEQREGWsFJGyfj"

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1m11t37h2bLU2wmd41aTEQREGWsFJGyfj
To: /kaggle/working/spoiler_y.txt
100%|████████████████████████████████████████| 277k/277k [00:00<00:00, 73.9MB/s]


In [6]:
# https://drive.google.com/file/d/1--PbeWSlq26rId939CNjGBWQV8Dcg6p5/view?usp=sharing
!gdown --id "1--PbeWSlq26rId939CNjGBWQV8Dcg6p5"

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1--PbeWSlq26rId939CNjGBWQV8Dcg6p5
To: /kaggle/working/spoiler_y_val.txt
100%|██████████████████████████████████████| 71.7k/71.7k [00:00<00:00, 73.4MB/s]


In [7]:
from transformers import BartTokenizer, BartForConditionalGeneration;
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer;

2024-07-03 18:25:21.519541: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 18:25:21.519639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 18:25:21.667342: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
import torch;
import torch.nn as nn;
import torch.optim as optim;
import torch.nn.functional as F;
from torch.utils.data import Dataset, DataLoader;

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu");
print(device);

cuda


In [10]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base');
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base');

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
class CustomDataset(nn.Module):
    def __init__(self,path_x,path_y,tokenizer,max_seq_length,max_target_length):
        super().__init__();

        self.max_seq_length = max_seq_length;
        self.max_target_length = max_target_length;

        with open(path_x,'r') as f:
            self.x = f.readlines();
        with open(path_y,'r') as f:
            self.y = f.readlines();

    def __len__(self):
        return len(self.x);

    def __getitem__(self,idx):
        x = tokenizer(self.x[idx],max_length=self.max_seq_length,truncation=True,padding='max_length',return_tensors='pt');
        y = tokenizer(self.y[idx],max_length=self.max_target_length,truncation=True,padding='max_length',return_tensors='pt');
        
        return {
            'input_ids':x['input_ids'].flatten(),
            'attention_mask':x['attention_mask'].flatten(),
            'labels':y['input_ids'].flatten(),
            'decoder_attention_mask':y['attention_mask'].flatten()
        };

In [12]:
training_dataset = CustomDataset(
    path_x='/kaggle/working/trainset_cmb.txt',
    path_y='/kaggle/working/spoiler_y.txt',
    tokenizer=tokenizer,
    max_seq_length=512,
    max_target_length=512
);

validation_dataset = CustomDataset(
    path_x='/kaggle/working/valset_cmb.txt',
    path_y='/kaggle/working/spoiler_y_val.txt',
    tokenizer=tokenizer,
    max_seq_length=512,
    max_target_length=512
);

In [13]:
args = Seq2SeqTrainingArguments(
    output_dir='/kaggle/working/',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
);

In [14]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
);

In [ ]:
trainer.train();

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,1.653580
2,No log,0.160972
3,No log,0.060730
4,No log,0.051267


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in 

In [ ]:
val_dataloader = DataLoader(validation_dataset,batch_size=4,shuffle=True);

In [ ]:
batch = next(iter(val_dataloader));

In [ ]:
output_text = model.generate(batch['input_ids'].to(device),max_length=300,num_beams=5,early_stopping=True);

In [ ]:
ids = 0;
txt = tokenizer.decode(output_text[ids],skip_special_tokens=True);
print("generated :-",txt);
print("original :- ",tokenizer.decode(batch['labels'][ids],skip_special_tokens=True));

In [ ]:
import os;
print(os.getcwd());
print();